In [4]:
#Import all the required libraries(install them if required.)
import json
from sseclient import SSEClient as EventSource
from collections import Counter
import datetime
import signal

#The url to fetch the Wikimedia data stream
url = ' https://stream.wikimedia.org/v2/stream/revision-create'

#The below function is used to handle interrupts and terminate the program 
def signal_handler(signal, frame):
    global interrupted
    interrupted = True

signal.signal(signal.SIGINT, signal_handler)


#This function is used to fetch the datat from the API
def fetch_data():
    pageList = {} #to store the pages being updated
    userReport = {} #to store the users who made changes to the "en.wikipedia.org" domain
    time1 = datetime.datetime.now()
    mins = 0 #to keep track of minutes
    global interrupted
    interrupted = False
    print("Generating reprot Please wait!..")
    #the fecting of data begins here
    for event in EventSource(url):
        if event.event == 'message':
            try:
                change = json.loads(event.data)            
            except ValueError:
                pass
            else:
                #as the data is received it is put into the required dictionary for further processing.
                if change["page_title"] not in pageList:
                    pageList[change["page_title"]] = change["meta"]["domain"] #the updated pages list
                if change["meta"]["domain"] == "en.wikipedia.org":
                    if change["performer"]["user_is_bot"] == False:
                        count = change["rev_len"]
                        userReport[change["performer"]["user_text"]] = count #the user list
                time2 = datetime.datetime.now()
                t = time2- time1
                #This is to manage time. Currently the code is set to 10 sec interval.
                if (t.seconds >= 10):
                    mins = mins+1
                    time1 = datetime.datetime.now()
                    print_func(pageList,userReport,mins)
                    #When an interrupt signal is received, terminate the program after completing the current cycle.
                    if interrupted:
                        print("\nYou have terminated the Program.Thank you!.")
                        break
    
#This function is used to sort the list based on the number of pages updated in the domain
def Convert(report):
    domainlist = list(report.items())
    domainlist.sort(key = lambda x: x[1])
    di = dict(domainlist)
    return di

#This function will display the report!
def print_func(pageList,userReport,mins):
    
    res = Counter(pageList.values())
    pageList = dict(res)
    sortedList=Convert(pageList)
    print("---------------------------------------------------------------------------------------------------------")
    print("\nReport for ",mins," minutes of data....")
    print("#####\t\t\tTotal number of Wikipedia Domains Updated:",len(sortedList))
    for ele in sortedList:
        print("\n",ele,":",sortedList[ele]," pages updated.")
    print("\n#####\t\t\tUsers who made changes to en.wikipedia.org are:\n")
    for ele in userReport:
        print("\n",ele,":",userReport[ele])
        
#This starts the program.
fetch_data()

Generating reprot Please wait!..
---------------------------------------------------------------------------------------------------------

Report for  1  minutes of data....
#####			Total number of Wikipedia Domains Updated: 26

 cs.wikipedia.org : 1  pages updated.

 sv.wikipedia.org : 1  pages updated.

 ja.wikipedia.org : 1  pages updated.

 et.wikipedia.org : 1  pages updated.

 fr.wiktionary.org : 1  pages updated.

 ta.wikipedia.org : 1  pages updated.

 lt.wikipedia.org : 1  pages updated.

 te.wikipedia.org : 1  pages updated.

 ha.wiktionary.org : 1  pages updated.

 de.wikipedia.org : 1  pages updated.

 meta.wikimedia.org : 1  pages updated.

 el.wikipedia.org : 1  pages updated.

 tr.wikipedia.org : 1  pages updated.

 zh.wikipedia.org : 2  pages updated.

 es.wikipedia.org : 2  pages updated.

 nl.wikipedia.org : 2  pages updated.

 ar.wikipedia.org : 2  pages updated.

 id.wikipedia.org : 2  pages updated.

 ru.wikipedia.org : 2  pages updated.

 uk.wikipedia.org : 2  pa